In [13]:
# NOTE: export SPARK_DRIVER_MEMORY=1g

In [14]:
import keras
from keras.applications import ResNet50
from keras.applications import imagenet_utils

* For resnet, we only support NCHW and Keras 1.2.2 for now

In [15]:
keras.backend.set_image_dim_ordering("th")

* Create ResNet50 using Keras

In [ ]:
keras_model = ResNet50(weights="imagenet")

* Init BigDL and spark environment

In [ ]:
from bigdl.keras.backend import *
from bigdl.transform.vision.image import *

redire_spark_logs()
show_bigdl_info_logs()
init_engine()
sparkConf = create_spark_conf().setAppName("test model")
sc = get_spark_context(sparkConf)

* Load a Keras definition and weights to BigDL.

In [ ]:
bmodel = DefinitionLoader.from_kmodel(keras_model)
WeightLoader.load_weights_from_kmodel(bmodel, keras_model)

* Load data into ImageFrame

In [ ]:
image_frame = ImageFrame.read("image-net", sc=sc)

* Define prepocess transformer

In [ ]:
transformer = Pipeline([BytesToMat(), Resize(256, 256),
                        CenterCrop(224, 224),
                        ChannelNormalize(123.68, 103.939, 116.779),
                        MatToTensor(), ImageFrameToSample()])

* Run prediction

In [ ]:
preds_frame = bmodel.predict_image(image_frame.transform(transformer))

* Show result of the first image

In [ ]:
preds = preds_frame.get_predict().take(1)

for pre in preds:
    path, pre_score = pre
    P = imagenet_utils.decode_predictions(np.expand_dims(pre_score, axis=0))
    print("Prediction for {}".format(path))
    # Display the top-5 prediction
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("    {}. {}: {:.2f}%".format(i + 1, label, prob * 100))

* While the origin image is
![scalars](image-net/158516268_796c3e047d.jpg)